In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hdf5plugin
import os

In [2]:
path_data = '/data/yyang18/database/SEA-AD/'

Load the raw data

In [3]:
data = sc.read_h5ad(os.path.join(path_data, 'MTG', 'RNAseq',
                                 'SEAAD_MTG_RNAseq_final-nuclei.2022-08-18.h5ad'))

In [4]:
data

AnnData object with n_obs × n_vars = 1378211 × 36601
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mo

save cell_information and cell_counts

In [5]:
cell_information = data.obs

In [6]:
# remove 5 reference samples, only keep cells from 84 aged patients
cell_information = cell_information[cell_information['Overall AD neuropathological Change']!='Reference']

In [7]:
cell_information.shape

(1240908, 133)

In [8]:
# The metadata of cells
cell_information.to_csv(os.path.join(path_data, 'out', 'cell_information.csv'),
                                     index=False)

In [9]:
# Count the number of cells in each cell type
cell_counts = cell_information[['Overall AD neuropathological Change','Subclass']].value_counts().rename_axis(['Overall AD neuropathological Change','Subclass']).reset_index(name='counts')
cell_counts = cell_counts.sort_values('Subclass')
cell_counts.to_csv(os.path.join(path_data, 'out', 'cell_count.csv'),
                                index=False)

save gene_list

In [10]:
gene_list = data.var

In [11]:
# The list of genes
gene_list.to_csv(os.path.join(path_data, 'out', 'gene_list.csv'),
                              index=False)

save expression_matrix by different cell types

In [12]:
expression_matrix = pd.DataFrame(data.X.toarray(), index=data.obs_names)

In [13]:
expression_matrix.columns = data.var_names

In [14]:
# remove reference samples, only keep cells from 84 aged patients
expression_matrix = expression_matrix[expression_matrix.index.isin(cell_information['sample_id'].to_list())]

In [15]:
expression_matrix.shape

(1240908, 36601)

In [27]:
# Save the gene expression data of each cell type separately
l_cell_type = []
l_data = []
n = 0
for name, group in cell_information[['Overall AD neuropathological Change', 'Subclass']].groupby('Subclass'):
    matrix = expression_matrix[expression_matrix.index.isin(group.index.to_list())]
    adata = sc.AnnData(X = matrix.to_numpy(), var = gene_list, obs = group)
    adata.write_h5ad(os.path.join(path_data, 'out', 'gene_expression',
                                  'cell_type_'+str(n)+'.h5ad'),
                     compression=hdf5plugin.FILTERS["zstd"],
                     compression_opts=hdf5plugin.Zstd(clevel=5).filter_options)
    l_cell_type.append(name)
    l_data.append('cell_type_'+str(n)+'.h5ad')
    n += 1

In [28]:
# Save the file about cell types and their h5ad filename and the number of expressed genes
h5ad = pd.DataFrame({'cell_type':l_cell_type, 
                     'data':l_data})
h5ad.to_csv(os.path.join(path_data, 'out', 'h5ad.csv'),
            index=False)